In [ ]:
import json
import numpy as np
import collections
import copy
from os import listdir
from os.path import isfile, join

In [ ]:
import findspark

findspark.init()
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().setAll(
    [
        ("spark.executor.memory", "8g"),
        ("spark.executor.cores", "2"),
        ("spark.executor.instances", "7"),
        ("spark.driver.memory", "32g"),
        ("spark.driver.maxResultSize", "10g"),
    ]
)
sc = SparkContext(conf=conf)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.types import Row
from pyspark.sql import SparkSession

spark = SparkSession(sc)

In [ ]:
from operator import add

In [ ]:
acsdb = sc.textFile(
    "/srv/samba/group_workspace_1/deng.595/workspace/table_transformer/data/ACSDb/coheader_stats_10092007_uniquedByDomain.txt"
)

In [ ]:
attr_stat = acsdb.filter(lambda x: x.startswith("single")).map(lambda x: x[7:].strip("_").split(" = "))
schema_stat = acsdb.filter(lambda x: x.startswith("combo")).map(lambda x: x[6:].strip("_").split(" = "))

In [ ]:
from itertools import combinations

In [ ]:
attr_stat = spark.createDataFrame(attr_stat.map(lambda x: (x[0], int(x[1]))), ["attribute", "freq"])
schema_stat = spark.createDataFrame(
    schema_stat.map(lambda x: (list(combinations(x[0].split("_"), 2)), int(x[1]))), ["attribute_pairs", "freq"]
).withColumn("schema_id", F.monotonically_increasing_id())

In [ ]:
schema_stat = schema_stat.select("freq", "schema_id", F.explode("attribute_pairs").alias("attribute_pair"))

In [ ]:
attr_co_occ = schema_stat.groupBy("attribute_pair").agg(F.sum("freq").alias("co_freq"))

In [ ]:
attr_co_occ.show()